In [1]:
import pandas as pd
import requests
import json
import os
import getpass
from pathlib import Path
import sqlite3
from time import sleep
import helpful_functions as hf
import ace_lib as ace
import pickle

In [2]:
s = ace.start_session()

Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_rskp5wrz6hNB9JKGddUt8tZ6JxBb



In [23]:
datasets_df = hf.get_datasets(s,region="GLB", universe="MINVOL1M", delay=1) # by default we load all datasets USA TOP3000 delay 1
datasets_df.head() # DataFrame.head() shows first 5 rows of the dataframe 

,id,name,description,category,subcategory,region,delay,universe,coverage,valueScore,userCount,alphaCount,fieldCount,pyramidMultiplier,themes,researchPapers
0,analyst11,ESG scores,Environmental Social Governance scores that ex...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-esg', 'name': 'ESG'}",GLB,1,MINVOL1M,0.7917,4.0,156,710,197,1.6,[],"[{'type': 'discussion', 'title': 'Getting star..."
1,analyst14,Estimations of Key Fundamentals,This dataset reports many items from financial...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-analyst-estimates', 'name': 'A...",GLB,1,MINVOL1M,0.5186,4.0,273,1285,926,1.6,[],"[{'type': 'discussion', 'title': 'Getting star..."
2,analyst15,Earnings forecasts,This dataset provides bottom-up forecast data ...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-analyst-estimates', 'name': 'A...",GLB,1,MINVOL1M,0.9929,2.0,503,3704,288,1.6,[],"[{'type': 'discussion', 'title': 'Getting star..."
3,analyst39,Analyst estimates & financial ratios,This dataset provides a lot of information on ...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-esg', 'name': 'ESG'}",GLB,1,MINVOL1M,1.0000,3.0,90,195,5,1.6,[],"[{'type': 'discussion', 'title': 'Getting star..."
4,analyst44,Integrated Broker Estimates,This dataset integrates the detail estimates f...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-crowdsourced-estimates', 'name...",GLB,1,MINVOL1M,0.8173,4.0,97,309,115,1.6,[],"[{'type': 'discussion', 'title': 'Getting star..."


In [24]:
datasets_df

,id,name,description,category,subcategory,region,delay,universe,coverage,valueScore,userCount,alphaCount,fieldCount,pyramidMultiplier,themes,researchPapers
0,analyst11,ESG scores,Environmental Social Governance scores that ex...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-esg', 'name': 'ESG'}",GLB,1,MINVOL1M,0.7917,4.0,156,710,197,1.6,[],"[{'type': 'discussion', 'title': 'Getting star..."
1,analyst14,Estimations of Key Fundamentals,This dataset reports many items from financial...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-analyst-estimates', 'name': 'A...",GLB,1,MINVOL1M,0.5186,4.0,273,1285,926,1.6,[],"[{'type': 'discussion', 'title': 'Getting star..."
2,analyst15,Earnings forecasts,This dataset provides bottom-up forecast data ...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-analyst-estimates', 'name': 'A...",GLB,1,MINVOL1M,0.9929,2.0,503,3704,288,1.6,[],"[{'type': 'discussion', 'title': 'Getting star..."
3,analyst39,Analyst estimates & financial ratios,This dataset provides a lot of information on ...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-esg', 'name': 'ESG'}",GLB,1,MINVOL1M,1.0000,3.0,90,195,5,1.6,[],"[{'type': 'discussion', 'title': 'Getting star..."
4,analyst44,Integrated Broker Estimates,This dataset integrates the detail estimates f...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-crowdsourced-estimates', 'name...",GLB,1,MINVOL1M,0.8173,4.0,97,309,115,1.6,[],"[{'type': 'discussion', 'title': 'Getting star..."
5,analyst46,Analyst Investment insight Data,The dataset incorporates the investment insigh...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-crowdsourced-estimates', 'name...",GLB,1,MINVOL1M,0.7703,4.0,55,140,5,1.6,[],"[{'type': 'research', 'title': 'Research Paper..."
6,analyst69,Fundamental Analyst Estimates,This dataset is a type of analyst dataset that...,"{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-analyst-estimates', 'name': 'A...",GLB,1,MINVOL1M,0.9369,2.0,539,3820,324,1.6,[],"[{'type': 'discussion', 'title': 'Getting star..."
7,fundamental13,Comprehensive Fundamentals Dataset,The dataset holds fundamental items included i...,"{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,0.3776,4.0,50,152,64,1.7,[],[]
8,fundamental17,Direct Fundamental Data,This dataset contains information about variou...,"{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,0.7816,3.0,471,2270,733,1.7,[],"[{'type': 'research', 'title': 'Research Paper..."
9,fundamental23,Fundamental Point in Time Data,This dataset gives point in time access to fun...,"{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",GLB,1,MINVOL1M,0.6704,3.0,476,3439,886,1.7,[],"[{'type': 'research', 'title': 'Research Paper..."


In [25]:
# select needed datasets
selected_datasets_df = datasets_df[
    (datasets_df["delay"] == 1) &
    (datasets_df["coverage"] > 0.8) & (datasets_df["coverage"] <= 1) &
    (datasets_df["fieldCount"] > 0) & (datasets_df["fieldCount"] < 1000) &
    (datasets_df["region"] == 'GLB') &
    (datasets_df["universe"] == 'MINVOL1M') &
    (datasets_df["userCount"] > 0) & (datasets_df["userCount"] < 100) &
    (datasets_df["valueScore"] > 1) & (datasets_df["valueScore"] < 10) 
    
    
].sort_values(by=['valueScore'], ascending=False)
len(selected_datasets_df)

8

In [28]:
dataset_id = selected_datasets_df.id.values.tolist() # create a list of selected datasets ids, our list has only one element
dataset_id

['analyst44',
 'univ1',
 'analyst39',
 'fundamental44',
 'model250',
 'pv29',
 'pv53',
 'risk60']

In [35]:
datafields_df = hf.get_datafields(s, dataset_id=dataset_id) # download all fields of dataset news
datafields_df.head()

""


In [36]:
datafields_df_vector = datafields_df[datafields_df["type"] == "VECTOR"]
datafields_df_matrix = datafields_df[datafields_df["type"] == "MATRIX"]

# Sort datafields by alphaCount in descending order
datafields_df_matrix = datafields_df_matrix.sort_values(by="alphaCount", ascending=False)
datafields_df_vector = datafields_df_vector.sort_values(by="alphaCount", ascending=False)

# Display the length of each datafield type
print(len(datafields_df_vector))
print(len(datafields_df_matrix))

KeyError: 'type'

In [20]:

li = datafields_df_matrix.id.values.tolist()

len(li)

114

In [21]:
expressions = [
   "ts_rank(ts_quantile({},252),252)+ts_rank(oth176_lmt_close_eq_score,252)"
]

In [22]:
count = 0
for i in li:
    for expr in expressions:
        try:
            count = count + 1
            print(count)
            if(count<0) : continue;
            simulate_data = {
                "type": "REGULAR",
                "settings": {
                    "instrumentType": "EQUITY",
                    "region": "GLB",
                    "universe": "MINVOL1M",
                    "delay": 1,
                    "decay": 0,
                    "neutralization": "SUBINDUSTRY",
                    "truncation": 0.08,
                    "pasteurization": "ON",
                    "unitHandling": "VERIFY",
                    "nanHandling": "OFF",
                    "language": "FASTEXPR",
                    "visualization": False,
                    "testPeriod": "P2Y0M"
                },
                "regular": expr.format(i)
            }
            
            simulate_response = s.post('https://api.worldquantbrain.com/simulations', json=simulate_data)
            print(simulate_data)
            print(simulate_response)
         
            simulation_progress_url = simulate_response.headers['Location']
            print(simulation_progress_url)
            finished = False
            
            while True:
                simulation_progress = s.get(simulation_progress_url)
              
                print('simulation_progress')
                if simulation_progress.headers.get("Retry-After", 0) == 0:
                    break
                sleep(float(simulation_progress.headers["Retry-After"]))
                
        except Exception as e:
            print(e)
        
 

1
{'type': 'REGULAR', 'settings': {'instrumentType': 'EQUITY', 'region': 'GLB', 'universe': 'MINVOL1M', 'delay': 1, 'decay': 0, 'neutralization': 'SUBINDUSTRY', 'truncation': 0.08, 'pasteurization': 'ON', 'unitHandling': 'VERIFY', 'nanHandling': 'OFF', 'language': 'FASTEXPR', 'visualization': False, 'testPeriod': 'P2Y0M'}, 'regular': 'ts_rank(ts_quantile(anl44_epsr_best_cur_fiscal_year_period,252),252)+ts_rank(oth176_lmt_close_eq_score,252)'}
<Response [201]>
https://api.worldquantbrain.com/simulations/1RwjkGgyo54a8xaQPbR88PG
simulation_progress
2
{'type': 'REGULAR', 'settings': {'instrumentType': 'EQUITY', 'region': 'GLB', 'universe': 'MINVOL1M', 'delay': 1, 'decay': 0, 'neutralization': 'SUBINDUSTRY', 'truncation': 0.08, 'pasteurization': 'ON', 'unitHandling': 'VERIFY', 'nanHandling': 'OFF', 'language': 'FASTEXPR', 'visualization': False, 'testPeriod': 'P2Y0M'}, 'regular': 'ts_rank(ts_quantile(anl44_roa_best_eeps_cur_yr,252),252)+ts_rank(oth176_lmt_close_eq_score,252)'}
<Response [20

KeyboardInterrupt: 